In [2]:
import torch
from functools import partial
from torch import nn
from timm.models.layers.drop import DropPath 

In [3]:
img = torch.rand(2,3,224,224)

In [4]:
patch_conv = nn.Conv2d(in_channels=3, out_channels=768, kernel_size=16, stride=16)

In [6]:
x = patch_conv(img)
print(x.shape)

torch.Size([2, 768, 14, 14])


In [8]:
x = x.flatten(2).transpose(1,2)
print(x.shape)

torch.Size([2, 196, 768])


In [11]:
layer_norm = partial(nn.LayerNorm,eps=1e-6)

In [12]:
norm1 = layer_norm(768)

In [50]:
x_ = x
x = norml(x)
print(x.shape)
B,N,C = x.shape
print(B)

torch.Size([2, 196, 768])
2


In [51]:
qkv_f = torch.nn.Linear(768,768*3,bias=False)

In [52]:
qkv = qkv_f(x)
print(qkv.shape)

torch.Size([2, 196, 2304])


In [53]:
qkv = qkv.reshape(B,N,C,-1)

qkv = qkv.permute(3,0,1,2)
print(qkv.shape)

torch.Size([3, 2, 196, 768])


In [54]:
q, k, v = qkv.unbind(0)
print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([2, 196, 768])
torch.Size([2, 196, 768])
torch.Size([2, 196, 768])


In [55]:
attn = (q@k.transpose(-2,-1))
print(attn.shape)

torch.Size([2, 196, 196])


In [56]:
attn = attn.softmax(dim=-1)

In [57]:
z = attn@v
print(z.shape)

torch.Size([2, 196, 768])


In [58]:
drop_path_rate = 0.1  #drop的概率
drop_path = DropPath(0.1)

In [59]:
x = x_ + drop_path(z)

In [63]:
norm2 = layer_norm(768)
x = norm2(x)

In [64]:
mlp_ratio = 4
fc1 = torch.nn.Linear(768,768*mlp_ratio)
act = nn.GELU()
dropout = nn.Dropout(0.1)

In [65]:
x = fc1(x)
x = act(x)
x= dropout(x)
print(x.shape)

torch.Size([2, 196, 3072])


In [66]:
fc2 = torch.nn.Linear(768*mlp_ratio,768)
x = fc2(x)
x = dropout(x)
x = x_ + drop_path(x)

In [67]:
print(x.shape)

torch.Size([2, 196, 768])
